In [1]:
import math

CN_NUM = "零壹贰叁肆伍陆柒捌玖"
CN_UNIT_INT = ["", "拾", "佰", "仟"]
CN_SECTION = ["", "万", "亿", "兆"]  # 可扩展
CN_UNIT_DEC = ["角", "分"]

def amount_to_rmb_cn(amount):
    """
    将数字金额转换为中文大写金额（到分）。
    例：1234567.89 -> 人民币壹佰贰拾叁万肆仟伍佰陆拾柒元捌角玖分
    规则：
      - 负数前加“负”
      - 四舍五入到分
      - 0 元返回“人民币零元整”
    """
    if amount is None or (isinstance(amount, float) and math.isnan(amount)):
        return ""

    # 尝试转为浮点
    try:
        num = float(amount)
    except Exception:
        # 无法解析为数字
        return ""

    sign = "负" if num < 0 else ""
    num = abs(num)

    # 四舍五入到分
    # 避免二进制浮点误差，转换为以分为单位的整数
    total_fen = int(round(num * 100))

    if total_fen == 0:
        return "人民币零元整"

    # 拆分整数分和小数分
    integer = total_fen // 100
    dec = total_fen % 100

    def four_digit_to_cn(n):
        """把 0-9999 转成中文，不含节单位（万/亿），保留内部零规则。"""
        res = []
        zero_flag = False
        for i in range(3, -1, -1):
            d = (n // (10**i)) % 10
            if d == 0:
                zero_flag = True
            else:
                if zero_flag and res:
                    res.append(CN_NUM[0])  # 插入一个“零”
                res.append(CN_NUM[d] + (CN_UNIT_INT[i] if i > 0 else ""))
                zero_flag = False
        # 去掉末尾的“零”
        s = "".join(res).rstrip(CN_NUM[0])
        return s or CN_NUM[0]

    # 处理整数部分，按 4 位一节
    if integer == 0:
        int_cn = "零"
    else:
        sections = []
        section_idx = 0
        need_zero = False  # 节之间的零
        while integer > 0:
            section_val = integer % 10000
            integer //= 10000
            if section_val == 0:
                need_zero = True
            else:
                part = four_digit_to_cn(section_val)
                # 在节之间添加“零”
                if need_zero and sections:
                    sections.append(CN_NUM[0])
                sections.append(part + CN_SECTION[section_idx])
                need_zero = False
            section_idx += 1
        int_cn = "".join(reversed(sections))

    # 处理小数部分（角分）
    if dec == 0:
        dec_cn = "整"
    else:
        jiao = dec // 10
        fen = dec % 10
        dec_parts = []
        if jiao > 0:
            dec_parts.append(CN_NUM[jiao] + CN_UNIT_DEC[0])
        if fen > 0:
            # 若角为 0 且有分，则需在前面补“零”
            if jiao == 0 and int_cn != "零":
                dec_parts.insert(0, CN_NUM[0])
            dec_parts.append(CN_NUM[fen] + CN_UNIT_DEC[1])
        dec_cn = "".join(dec_parts)

    # 组合
    # 特例：整数为零但有小数，如 0.05 -> 人民币零元伍分（不需要额外“零”）
    result = f"美元{sign}{int_cn}元{dec_cn}"
    # 规范一些可能产生的重复零
    result = result.replace("零零", "零").replace("零元整", "零元整")
    return result

In [2]:
import pandas as pd
import math

# 粘贴 amount_to_rmb_cn

input_path = r'C:\Users\CTIG\Downloads\001 进口烟叶合同制作\合同号.xlsx'
sheet_name = "Sheet1"   # 0 表示第一个工作表；或填字符串表名
amount_col_name = "#销售金额#"
new_col_name = "#中文金额#"

df = pd.read_excel(input_path, sheet_name=sheet_name)

def to_cn_safe(x):
    return amount_to_rmb_cn(x)

df[new_col_name] = df[amount_col_name].apply(to_cn_safe)

# 保留原样式需用 openpyxl，pandas 会丢样式；这里只保存数据
df.to_excel("output.xlsx", index=False)
print("完成")

完成
